[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nmickevicius/MCW_BIOP_03-238_MRI/blob/lecture02/02_fourier_transforms/fourier_transforms.ipynb)

In [1]:
# %% Imports (stick to your requested set)
%matplotlib inline
import numpy as np
from ipywidgets import interact, interactive, FloatSlider, IntSlider, Checkbox
from IPython.display import clear_output, display, HTML

# Extra import for plotting
import matplotlib.pyplot as plt

# --- Fourier-series boxcar demo ---------------------------------------------

def _boxcar_periodic(x, w, phi0):
    """
    Periodic boxcar on [-pi, pi] with half-width w (0<w<=pi) and center shift phi0.
    Value 1 inside the pulse and 0 outside, repeated every 2*pi.
    """
    # Wrap (x - phi0) to [-pi, pi]
    y = (x - phi0 + np.pi) % (2*np.pi) - np.pi
    return (np.abs(y) <= w).astype(float)

def _series_coeffs_boxcar(w, phi0, K):
    """
    Fourier series coefficients for the shifted boxcar on [-pi, pi].
    Start from the centered even boxcar (only cosines), then shift by phi0.

    For the centered pulse:
      a0 = 2w/pi
      a_k0 = (2/pi) * sin(k w) / k
      b_k0 = 0

    After shift by phi0:
      a_k = a_k0 * cos(k*phi0)
      b_k = a_k0 * sin(k*phi0)
    """
    k = np.arange(1, K+1, dtype=float)
    a0 = 2*w/np.pi
    a_k0 = (2/np.pi) * np.sin(k*w) / k
    a_k = a_k0 * np.cos(k*phi0)
    b_k = a_k0 * np.sin(k*phi0)
    return a0, a_k, b_k

def boxcar_fourier_demo(N=12, w_over_pi=0.40, phi0=0.0, show_harmonics=True):
    """
    Re-rendered on each slider move (works with %matplotlib inline).
    N: number of harmonics
    w_over_pi: half-width as fraction of pi (0<w/pi<1)
    phi0: shift of the pulse center (radians)
    """
    clear_output(wait=True)

    # Grid over one period
    x = np.linspace(-np.pi, np.pi, 2000, endpoint=False)

    # Parameters
    w = np.clip(w_over_pi, 1e-3, 0.999) * np.pi
    K = int(max(1, N))

    # Target periodic boxcar
    f = _boxcar_periodic(x, w, phi0)

    # Coefficients and partial sum
    a0, a_k, b_k = _series_coeffs_boxcar(w, phi0, K)
    k = np.arange(1, K+1, dtype=float)

    # Vectorized partial sum: a0/2 + sum_k [a_k cos(kx) + b_k sin(kx)]
    s = (a0 / 2.0) + (a_k @ np.cos(np.outer(k, x))) + (b_k @ np.sin(np.outer(k, x)))

    # Plot
    plt.figure(figsize=(7, 5))
    ax = plt.gca()
    ax.set_title("Fourier Series Approximation of a Boxcar (period $2\\pi$)")
    ax.set_xlabel("x")
    ax.set_ylabel("amplitude")
    ax.set_xlim(-np.pi, np.pi)
    ax.set_ylim(-0.3, 1.3)
    ax.grid(True, linestyle=":")

    # Boxcar edges for reference
    ax.axvline((phi0 - w + np.pi) % (2*np.pi) - np.pi, linestyle="--", linewidth=1)
    ax.axvline((phi0 + w + np.pi) % (2*np.pi) - np.pi, linestyle="--", linewidth=1)

    # Target and partial sum
    ax.plot(x, f, linewidth=2, label="boxcar")
    ax.plot(x, s, linewidth=2, label=f"partial sum (N={K})")

    # Optional: show individual harmonics in amplitude–phase form
    if show_harmonics:
        # A_k = sqrt(a_k^2 + b_k^2) ; phi_k = atan2(b_k, a_k)
        A_k = np.sqrt(a_k**2 + b_k**2)
        phi_k = np.arctan2(b_k, a_k)

        # To avoid clutter, plot up to 30 faded harmonics
        max_to_plot = min(K, 30)
        for kk in range(1, max_to_plot+1):
            term = A_k[kk-1] * np.cos(kk*x - phi_k[kk-1])
            ax.plot(x, term, linewidth=0.8, alpha=0.35)

        if K > max_to_plot:
            ax.text(0.02, 0.04, f"… plus {K - max_to_plot} more harmonics",
                    transform=ax.transAxes)

    ax.legend(loc="upper right")
    plt.tight_layout()
    plt.show()

    # Small readout for coefficients (first few)
    shown = min(6, K)
    A = np.sqrt(a_k[:shown]**2 + b_k[:shown]**2)
    PH = np.arctan2(b_k[:shown], a_k[:shown])
    rows = [
        "<tr><th>k</th><th>A_k</th><th>ϕ_k (rad)</th></tr>"
    ] + [
        f"<tr><td>{i}</td><td>{A[i-1]:.4g}</td><td>{PH[i-1]:.4g}</td></tr>"
        for i in range(1, shown+1)
    ]
    html = f"""
    <div style="font-family: ui-sans-serif, system-ui; line-height:1.35;">
      <b>Series summary:</b>
      a₀/2 = {a0/2:.4g},  w/π = {w_over_pi:.3f},  shift φ₀ = {phi0:.3f} rad
      <table style="margin-top:6px;border-collapse:collapse" border="1" cellpadding="4">
        {''.join(rows)}
      </table>
      <div style="margin-top:6px; font-size: 90%;">
        Note: shifting the pulse by φ₀ mixes cosine/sine terms:
        a_k = a_k⁰ cos(kφ₀), b_k = a_k⁰ sin(kφ₀), so each harmonic appears as
        A_k cos(kx − ϕ_k).
      </div>
    </div>
    """
    display(HTML(html))

# --- UI: sliders and toggle --------------------------------------------------

interact(
    boxcar_fourier_demo,
    N=IntSlider(value=12, min=1, max=60, step=1, description="Harmonics N"),
    w_over_pi=FloatSlider(value=0.40, min=0.05, max=0.95, step=0.01, description="Half-width w/π"),
    phi0=FloatSlider(value=0.0, min=-3.1416, max=3.1416, step=0.01, description="Shift φ₀ (rad)"),
    show_harmonics=Checkbox(value=True, description="Show individual harmonics"),
)


interactive(children=(IntSlider(value=12, description='Harmonics N', max=60, min=1), FloatSlider(value=0.4, de…

<function __main__.boxcar_fourier_demo(N=12, w_over_pi=0.4, phi0=0.0, show_harmonics=True)>